In [1]:
import gc

gc.enable()
gc.collect()

0

In [2]:
import nltk
import pandas as pd

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.tokenize import word_tokenize

In [ ]:
data = pd.read_csv("~/Sentiment Analysis Dataset.csv",usecols=('Sentiment','SentimentText'),encoding='utf-8')
amazon_data = pd.read_csv("~/amazon_cells_labelled.txt",delimiter="\t",header=None,names=['SentimentText','Sentiment'],encoding='utf-8')
imdb_data = pd.read_csv("~/imdb_labelled.txt",delimiter="\t",header=None,names=['SentimentText','Sentiment'],encoding='utf-8')
yelp_data = pd.read_csv("~/yelp_labelled.txt",delimiter="\t",header=None,names=['SentimentText','Sentiment'],encoding='utf-8')

In [ ]:
print("data \n", data.info())
print("amzon data \n", amazon_data.info())
print('imdb ', imdb_data.info())
print('yelp data ', yelp_data.info())

In [ ]:
duplicate_data=data[:200000]
duplicate_data.info()
duplicate_data=pd.concat([duplicate_data, amazon_data,imdb_data,yelp_data], ignore_index=True)
print(duplicate_data.info())

In [ ]:
duplicate_data.head()

In [ ]:
duplicate_data = duplicate_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
import numpy as np
import re

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [ ]:
duplicate_data['tidy_sent'] = np.vectorize(remove_pattern)(duplicate_data['SentimentText'], "@[\w]*")

In [ ]:
duplicate_data['tidy_sent'] = duplicate_data['tidy_sent'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
duplicate_data['tidy_sent'] = duplicate_data['tidy_sent'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
duplicate_data.head()

In [ ]:
tokenized_sent = duplicate_data['tidy_sent'].apply(lambda x: x.split())

In [ ]:
tokenized_sent.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

tokenized_sent = tokenized_sent.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

In [ ]:
for i in range(len(tokenized_sent)):
    tokenized_sent[i] = ' '.join(tokenized_sent[i])

duplicate_data['tidy_sent'] = tokenized_sent

In [ ]:
duplicate_data.head()

In [ ]:
import pickle
from nltk.tokenize import word_tokenize

In [ ]:
all_words=[]
for row in duplicate_data.tidy_sent:
    all_words.append(word_tokenize(row))

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

In [ ]:
allwords = ' '.join([text for text in duplicate_data['tidy_sent']])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(allwords)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
pos_words = ' '.join([text for text in duplicate_data['tidy_sent'][duplicate_data['Sentiment'] == 1]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(pos_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
neg_words = ' '.join([text for text in duplicate_data['tidy_sent'][duplicate_data['Sentiment'] == 0]])

wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(neg_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
duplicate_data_f = open("~/duplicate_data.pickle", "rb")
duplicate_data = pickle.load(duplicate_data_f)
duplicate_data_f.close()

In [ ]:
temp=duplicate_data[['tidy_sent','sentiment_word']]

In [ ]:
temp.rename(columns={'tidy_sent':'text','sentiment_word':'sentiment'},inplace=True)
temp.head()

In [ ]:
from nltk.tokenize import word_tokenize
all_words = []
allowed_types=["JJ","JJR","JJS","RB","RBS","RBR","VB","VBD","VBG","VBN","VBP","VBZ","UH"]
for row in temp.text:
    #words=word_tokenize(row)
    tag=nltk.pos_tag(word_tokenize(row))
    for w in range(len(tag)):
        if tag[w][1] in allowed_types:
            all_words.append(tag[w][0].lower())

In [ ]:
keys=list(all_words.keys())

In [ ]:
save_word_features = open("~/word_features.pickle","wb")
pickle.dump(keys, save_word_features)
save_word_features.close()

In [ ]:
te=temp.values.tolist()

In [ ]:
word_features = list(all_words.keys())
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in range(len(words)):
        features[words[w]]=(words[w] in word_features)        
    return features
featuresets = [(find_features(rev), category) for (rev, category) in te]

In [ ]:
save_dataset = open("~/featuresets.pickle","wb")
pickle.dump(featuresets, save_dataset)
save_dataset.close()
t.to_csv("~/featuresets(209824).csv")

In [ ]:
import random
random.shuffle(featuresets1)

In [ ]:
training_set = featuresets
testing_set = featuresets[209800:]

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
#classifier.show_most_informative_features(15)
save_classifier = open(r"~\processed data\originalnaivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()
del classifier

In [ ]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)
save_classifier = open(r"~\processed data\LinearSVC_classifier.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()
del LinearSVC_classifier

In [ ]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)
save_classifier = open(r"~\processed data\LogisticRegression_classifier.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()
del LogisticRegression_classifier

In [ ]:
dataset=pd.DataFrame(featuresets)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(dataset['text'].values.astype('U'))

In [ ]:
bow_f = open("~/processed data/bow.pickle", "rb")
bow = pickle.load(bow_f)
bow_f.close()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow[:209800]
test_bow = bow[209800:]

In [ ]:
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, dataset.senti[:209800], test_size=0.3)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(xtrain_bow, ytrain)
pred = clf.predict(xtrain_bow)
print('classification report \n',classification_report(ytrain, pred))

In [ ]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int_)

f1_score(pd.to_numeric(yvalid), prediction_int)

In [ ]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int_)

f1_score(pd.to_numeric(yvalid), prediction_int)

In [ ]:
save_classifier = open(r"~\processed data\LogisticRegression.pickle","wb")
pickle.dump(lreg, save_classifier)
save_classifier.close()

In [ ]:
svclassifier = SVC(kernel='poly', degree=8)  
svclassifier.fit(xtrain_bow, ytrain)  
y_pred = svclassifier.predict(xvalid_bow)  
print(confusion_matrix(yvalid, y_pred))  
print(classification_report(yvalid, y_pred)) 

In [ ]:
save_classifier = open(r"~\processed data\svcclassifier.pickle","wb")
pickle.dump(svclassifier, save_classifier)
save_classifier.close()

In [ ]:
randomforest_classifier = RandomForestClassifier()
randomforest_classifier.fit(xtrain_bow, ytrain)
pred= randomforest_classifier.predict(xtrain_bow)
print('classification report \n',classification_report(ytrain, pred))  
#f1_score(pd.to_numeric(yvalid), prediction_int)

In [ ]:
save_classifier = open(r"~\processed data\Randomforestclassifier.pickle","wb")
pickle.dump(randomforest_classifier, save_classifier)
save_classifier.close()
del randomforest_classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(xtrain_bow, ytrain)
pred = clf.predict(xtrain_bow)
print('classification report \n',classification_report(ytrain, pred))

In [ ]:
save_classifier = open(r"~\processed data\DecisionTreeClassifier.pickle","wb")
pickle.dump(DecisionTreeClassifier, save_classifier)
save_classifier.close()